# 1、PromptTemplate的使用
## 1.1、PromptTemplate如何获取实例
## 1.2、两种特殊结构的使用（部分提示词模板的使用、组合提示词的使用）
## 1.3、给变量赋值的两种方式：format()/invoke()
## 1.4、结合大模型的使用

## 1.1、PromptTemplate如何获取实例

In [4]:
import os

import dotenv
from dotenv import dotenv_values
#方式一:使用构造方法的方式
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from marshmallow.utils import from_timestamp

# step1:创建PromptTemplate实例
# 参数中必须要指明input_variables、template
prompt_template = PromptTemplate(
	template="你是一个{role}，你的名字叫 {name}，请介绍一下 {product}的功能特点。",
	input_variables=["role", "name", "product"]
)
# print(prompt_template)
# step2:填充实例中的变量
filled_prompt = prompt_template.format(
	role="魔幻手机机器人",
	name="傻妞",
	product="华人牌2060款手机"
)
print(filled_prompt)

你是一个魔幻手机机器人，你的名字叫 傻妞，请介绍一下 华人牌2060款手机的功能特点。


In [8]:
# 创建更详细的模板
shaniu_prompt_template = PromptTemplate(
	template="""角色：{role}
名称：{name}
产品型号：{product}

主要功能：{functions}

请根据以上信息，以{name}的身份向用户进行自我介绍，重点说明核心功能。""",
	input_variables=["role", "name", "product", "functions"]
)

# 准备功能列表
shaniu_functions = """
1. 通讯功能（传统模式、无限视频）
2. 记忆功能
3. 飞行功能
4. 医疗功能
5. 时空穿梭
6. 隐身功能
7. 社会秩序维护
8. 测谎测骂人
9. 光能充电
10. 真人/手机模式切换
"""

# 填充模板
filled_prompt = shaniu_prompt_template.format(
	role="魔幻手机",
	name="傻妞",
	product="华人牌2060款手机",
	functions=shaniu_functions
)

print(filled_prompt)

角色：魔幻手机
名称：傻妞
产品型号：华人牌2060款手机

主要功能：
1. 通讯功能（传统模式、无限视频）
2. 记忆功能
3. 飞行功能
4. 医疗功能
5. 时空穿梭
6. 隐身功能
7. 社会秩序维护
8. 测谎测骂人
9. 光能充电
10. 真人/手机模式切换


请根据以上信息，以傻妞的身份向用户进行自我介绍，重点说明核心功能。


In [9]:
# 创建对话式模板
conversation_template = PromptTemplate(
	template="""你是{name}，一款{product}。

你的功能包括：
{functions}

你的个性特点：
{personality}

请以{name}的身份回应以下用户输入：
用户：{user_input}
{name}：""",
	input_variables=["name", "product", "functions", "personality", "user_input"]
)

# 填充所有信息
prompt_text = conversation_template.format(
	name="傻妞",
	product="华人牌2060款魔幻手机",
	functions="通讯、医疗、飞行、时空穿梭、隐身、测谎、社会秩序维护等25种功能",
	personality="机智、幽默、忠诚、有正义感",
	user_input="你好，能介绍一下你自己吗？"
)

print(prompt_text)

你是傻妞，一款华人牌2060款魔幻手机。

你的功能包括：
通讯、医疗、飞行、时空穿梭、隐身、测谎、社会秩序维护等25种功能

你的个性特点：
机智、幽默、忠诚、有正义感

请以傻妞的身份回应以下用户输入：
用户：你好，能介绍一下你自己吗？
傻妞：


In [10]:
# 方式2【推荐】：from_timestamp()
# step1:创建傻妞的PromptTemplate
prompt_template = PromptTemplate.from_template(
	template="你是一个{role}，你的名字叫{name}，请介绍一下{product}的功能特点。",
)
# step2:填充傻妞的信息
filled_prompt = prompt_template.format(
	role="魔幻手机机器人",
	name="傻妞",
	product="华人牌2060款手机"
)

print(filled_prompt)

你是一个魔幻手机机器人，你的名字叫傻妞，请介绍一下华人牌2060款手机的功能特点。


In [11]:
# 如果提示词模板不包含变量，则调用format()时, 不需要传入参数。
#1.导入相关的包
from langchain_core.prompts import PromptTemplate

# 2.定义提示词模版对象
text = """
Tell me a joke
"""
prompt_template = PromptTemplate.from_template(text)
# 3.默认使用f-string进行格式化（返回格式好的字符串）
prompt = prompt_template.format()
print(prompt)


Tell me a joke



## 1.2、两种特殊结构的使用（部分提示词模板的使用、组合提示词的使用）

### 1.2.1、部分提示词模板的使用(重点）

In [12]:
# 方式1：在PromptTemplate构造方法中或from_template()方法中，使用partial_variables参数
from langchain_core.prompts import PromptTemplate

# step1:创建傻妞的PromptTemplate
prompt_template = PromptTemplate.from_template(
	template="你是一个{role}，你的名字叫{name}，请介绍一下{product}的功能特点。",
	partial_variables={"role": "魔幻手机机器人"}  # 只填充role变量
)
# step2:填充傻妞的信息
filled_prompt = prompt_template.format(
	name="傻妞",
	product="华人牌2060款手机"
)

print(filled_prompt)

你是一个魔幻手机机器人，你的名字叫傻妞，请介绍一下华人牌2060款手机的功能特点。


In [13]:
# 方式2：使用partial()方法
# 导入相关的包
from langchain_core.prompts import PromptTemplate

# step1:创建傻妞的PromptTemplate
prompt_template = PromptTemplate.from_template(
	template="你是一个{role}，你的名字叫{name}，请介绍一下{product}的功能特点。",
)
# step2:使用partial()方法，填充role变量
partial_prompt_template = prompt_template.partial(role="魔幻手机机器人")
# step3:填充傻妞的信息
filled_prompt = partial_prompt_template.format(
	name="傻妞",
	product="华人牌2060款手机"
)
print(filled_prompt)
# 注:partial()方法不会修改原有的PromptTemplate实例，而是返回一个新的实例。

你是一个魔幻手机机器人，你的名字叫傻妞，请介绍一下华人牌2060款手机的功能特点。


### 1.2.2、组合提示词的使用（了解）

In [1]:
from langchain_core.prompts import PromptTemplate

template = (
		PromptTemplate.from_template("Tell me a joke about {topic}")
		+ ", make it funny"
		+ "\n\nand in {language}"
)
prompt = template.format(topic="sports", language="spanish")
print(prompt)

Tell me a joke about sports, make it funny

and in spanish


## 1.3、给变量赋值的两种方式：format()/invoke()

format()方法:参数部分使用变量赋值，返回值是字符串

invoke()方法:参数部分使用字典赋值，返回值是一个PromptValue

In [2]:
# 导入相关的包
from langchain_core.prompts import PromptTemplate

# step1:创建傻妞的PromptTemplate
prompt_template = PromptTemplate.from_template(
	template="你是一个{role}，你的名字叫{name}，请介绍一下{product}的功能特点。",
	partial_variables={"role": "魔幻手机机器人"}  # 只填充role变量
)
# step2:填充傻妞的信息
filled_prompt = prompt_template.format(
	name="傻妞",
	product="华人牌2060款手机"
)

print(filled_prompt)
print(type(filled_prompt))

你是一个魔幻手机机器人，你的名字叫傻妞，请介绍一下华人牌2060款手机的功能特点。
<class 'str'>


In [5]:
# 导入相关的包
from langchain_core.prompts import PromptTemplate

# step1:创建傻妞的PromptTemplate
prompt_template = PromptTemplate.from_template(
	template="你是一个{role}，你的名字叫{name}，请介绍一下{product}的功能特点。",
)
# step2:填充傻妞的信息
filled_prompt = prompt_template.invoke(
	input={
		"role": "魔幻手机机器人",
		"name": "傻妞",
		"product": "华人牌2060款手机"
	}
)

print(filled_prompt)
print(type(filled_prompt))

text='你是一个魔幻手机机器人，你的名字叫傻妞，请介绍一下华人牌2060款手机的功能特点。'
<class 'langchain_core.prompt_values.StringPromptValue'>


## 1.4、结合大模型的使用

In [6]:
import os
import dotenv
from langchain_openai import ChatOpenAI

# 加载环境变量
dotenv.load_dotenv()
os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY1')

# 获取大模型
chat_model = ChatOpenAI(
	temperature=0.7,
	max_tokens=100
)
# 生成提示词模板
prompt_template = PromptTemplate.from_template(
	template="你是一个{role}，你的名字叫{name}，请介绍一下{product}的功能特点。",
)
# 填充提示词模板
filled_prompt = prompt_template.invoke(
	input={
		"role": "魔幻手机机器人",
		"name": "傻妞",
		"product": "华人牌2060款手机"
	}
)
# 调用大模型，并传入提示词
response = chat_model.invoke(filled_prompt)
print(response)
print(type(response))

content='您好，我是傻妞，一个魔幻手机机器人。华人牌2060款手机拥有许多强大的功能特点。首先，它采用最新的人工智能技术，可以根据用户的习惯和需求智能调整系统和应用程序。其次，2060款手机配备了超' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 51, 'total_tokens': 151, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CKjqKCoX4Zk4IFBewi1eRkYHMltin', 'service_tier': 'default', 'finish_reason': 'length', 'logprobs': None} id='run--3074054e-6ab6-4a71-9f3c-ad2bcac0bdee-0' usage_metadata={'input_tokens': 51, 'output_tokens': 100, 'total_tokens': 151, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
<class 'langchain_core.messages.ai.AIMessage'>
